In [72]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import copy
import pandas as pd
import time
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [73]:
import sys
import os
sys.path.append(os.path.abspath('../utils'))
#from ae_torch_classes import *
from data_loading_paths import *
#from utils.subspace_clustering_helper_funcs import *

In [74]:
#from utils.data_loading_paths import *

kai_laptop = False
brc_desktop = True

if kai_laptop:
    data_path = kai_data_path
    model_dir_path = kai_model_dir_path
    metadata_path = kai_metadata_path
elif brc_desktop:
    data_path = brc_data_path
    model_dir_path = brc_model_dir_path
    metadata_path = brc_metadata_path

metadata_cols_df = pd.read_pickle(metadata_path)

emg_training_users_df = pd.read_pickle(data_path+emg_dir+'training_users_df.pkl').drop(metadata_cols, axis=1)
emg_test_users_df = pd.read_pickle(data_path+emg_dir+'test_users_df.pkl').drop(metadata_cols, axis=1)

both_training_users_df = pd.read_pickle(data_path+both_dir+'training_users_df.pkl').drop(metadata_cols, axis=1)
both_test_users_df = pd.read_pickle(data_path+both_dir+'test_users_df.pkl').drop(metadata_cols, axis=1)

undstand_emg_df = pd.read_pickle(data_path+'filtered_datasets\\metadata_EMG_allgestures_allusers.pkl').drop(metadata_cols, axis=1)
#undstand_both_df = pd.read_pickle(data_path+'filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl').drop(metadata_cols, axis=1)

full_emg_users_df = pd.concat([emg_training_users_df, emg_test_users_df])
full_both_users_df = pd.concat([both_training_users_df, both_test_users_df])


In [75]:
data_df = pd.concat([metadata_cols_df, undstand_emg_df], axis=1)

print(data_df.shape)
data_df.head()

(204800, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000004,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000005,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000004,0.000005,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.000002,0.000002,0.000002,0.000002,0.000005,0.000006,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.000003,0.000002,0.000002,0.000002,0.000005,0.000007,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


## Filter out P111 P124 and P131 and unimpared participants

In [76]:
filtered_df = data_df[~data_df['Participant'].isin(['P004', 'P005', 'P006', 'P008', 'P010', 'P011', 'P111','P124','P131'])]
print(filtered_df.shape)
filtered_df.head()

(147200, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000004,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000003,0.000005,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.000002,0.000002,0.000001,0.000002,0.000004,0.000005,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.000002,0.000002,0.000002,0.000002,0.000005,0.000006,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.000003,0.000002,0.000002,0.000002,0.000005,0.000007,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


In [77]:
print(len(filtered_df['Participant'].unique()))

23


## Standardization

In [78]:
standardize = "EMG"  # "True", "standardize_kais_built_in", "ben_bad_by_channel"

if standardize==True or standardize=="True":
    # FOR BOTH BIOSIGNALS!
    IMU_df = filtered_df.iloc[:, 3:75]
    EMG_df = filtered_df.iloc[:, 75:91]
    IMU_mean = IMU_df.mean()
    EMG_mean = EMG_df.mean()
    IMU_sd = IMU_df.values.std()
    EMG_sd = EMG_df.values.std()
    IMU_scaled = pd.DataFrame((IMU_df - IMU_mean) / IMU_sd)
    EMG_scaled = pd.DataFrame((EMG_df - EMG_mean) / EMG_sd)
    scaled_signal_df = pd.concat([IMU_scaled, EMG_scaled], axis=1)
elif standardize=="EMG":
    EMG_df = filtered_df.iloc[:, 3:]
    EMG_mean = EMG_df.mean()
    EMG_sd = EMG_df.values.std()
    EMG_scaled = pd.DataFrame((EMG_df - EMG_mean) / EMG_sd)
    scaled_signal_df = EMG_scaled
elif standardize=="standardize_kais_built_in":
    scaled_signal_df = preprocess_df_by_gesture(filtered_df.iloc[:,3:], '$B', biosignal_switch_ix_lst=[72], trial_length=64)
elif standardize=="ben_bad_by_channel":
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaled_signal_df = pd.DataFrame(scaler.fit_transform(filtered_df.iloc[:,3:]), columns=filtered_df.columns[3:91])
else:
    raise ValueError()
    scaled_signal_df = filtered_df.iloc[:, 3:]

print(scaled_signal_df.shape)
scaled_signal_df.head()

(147200, 16)


,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,-0.157965,-0.056802,-0.140806,-0.592281,-0.060937,-0.008374,-0.079964,-0.424846,-0.341147,0.390614,-0.265030,-0.480894,-0.216967,0.024762,-0.467114,-0.396594
1,-0.155577,-0.051186,-0.140692,-0.583632,-0.045769,0.027750,-0.047851,-0.400002,-0.328843,0.167281,-0.142393,-0.357103,-0.188265,0.353845,-0.463692,-0.384817
2,-0.161824,-0.048465,-0.144073,-0.568829,-0.013863,0.046856,-0.035608,-0.394155,-0.297259,-0.114515,-0.253313,-0.429419,-0.173898,0.441633,-0.415650,-0.375483
3,-0.158830,-0.049079,-0.138959,-0.566229,0.021487,0.092730,-0.033571,-0.401859,-0.245228,-0.179894,-0.262888,-0.492773,-0.140662,0.266036,-0.408565,-0.333815
4,-0.132961,-0.048187,-0.123925,-0.568090,0.030413,0.112800,-0.021838,-0.410278,-0.158180,-0.257946,-0.168307,0.245196,-0.140031,0.353777,-0.447436,-0.339644


## Perform PCA

In [79]:
pca = PCA(n_components=3)
#pca_by_signal_result_df = pd.DataFrame(data=pca.fit_transform(scaled_signal_df), columns=['PC1', 'PC2', 'PC3'])
#pca_by_column_result_df = pd.DataFrame(data=pca.fit_transform(scaled_column_df), columns=['PC1', 'PC2', 'PC3'])
pca_df = pd.DataFrame(data=pca.fit_transform(scaled_signal_df), columns=['PC1', 'PC2', 'PC3']).reset_index(drop=True)
filtered_df = filtered_df.reset_index(drop=True)
#pca_by_signal_result_df = pca_by_signal_result_df.reset_index(drop=True)
#pca_by_column_result_df = pca_by_column_result_df.reset_index(drop=True)
#pca_by_signal_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_signal_result_df], axis=1)
#pca_by_column_df = pd.concat([filtered_df.iloc[:,0:3], pca_by_column_result_df], axis=1)
pca_df = pd.concat([filtered_df.iloc[:,:3], pca_df], axis=1)

print(pca_df.shape)
pca_df.head()


(147200, 6)


,Participant,Gesture_ID,Gesture_Num,PC1,PC2,PC3
0,P102,pan,1,-0.673765,-0.584377,-0.383000
1,P102,pan,1,-0.633114,-0.448132,-0.304246
2,P102,pan,1,-0.598663,-0.588107,-0.295491
3,P102,pan,1,-0.586446,-0.647375,-0.310281
4,P102,pan,1,-0.548038,0.047679,-0.439862


## Plot PCA

In [ ]:
fig = px.scatter_3d(pca_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Gesture_ID', 
                    title='3D PCA of EMG+IMU, Colored by Gesture')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed

fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Participant', 
                    title='3D PCA of EMG+IMU, Colored by Participant')

fig.update_traces(marker=dict(size=3))  # Adjust marker size as needed
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Add Handedness 

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
# Read all sheets into a dictionary of dataframes
all_sheets_dict = pd.read_excel("C:\\Users\\rubin\\Research\\user_gesture_descriptions_qualitative_analysis.xlsx", sheet_name=None)

# Initialize an empty list to hold individual dataframes
df_list = []

# Iterate through the dictionary and process each sheet
for user_id, df in all_sheets_dict.items():
    df.columns.values[1] = 'gesture'
    # Identify columns with "Unnamed" in their names
    unnamed_cols = df.columns.str.contains("Unnamed")
    # Drop those columns
    df = df.loc[:, ~unnamed_cols]
    df = df.iloc[:10,:]
    df= df.reset_index(drop=True)
    df_list.append(df)  # Add the dataframe to the list

In [18]:
print(df_list[2].shape)
df_list[2].head()

(10, 17)


,PID,gesture,time,description,hand used,lifted arms off arm rest?,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,agreement,midair or on table,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1
0,P103,rotate,3,make a circle with right hand without moving f...,right,barely,yes,upper-body (bicep/tricep),shoulder,NaN,rotate from shoulders,midair,what I can do with my abilities,NaN,NaN,due to handedness,NaN
1,P103,move,2,move right hand from left to right without rai...,right,barely,yes,upper-body (bicep/tricep),NaN,NaN,point and move entire hand in 2D (like on a fl...,on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
2,P103,select-single,2,using a mouse to select,right,barely,yes,shoulder,upper-body (bicep/tricep),NaN,point at table (like clicking a mouse),on a 2D plane but not on table,NaN,NaN,NaN,NaN,NaN
3,P103,zoom-in,3,move head forward and backwards,head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN
4,P103,zoom-out,3,"move head backwards, then forward",head,no,yes,shoulder,NaN,NaN,move head closer/away to screen,neither,NaN,NaN,NaN,NaN,NaN


In [19]:
# Concatenate all DataFrames in the list into one large DataFrame
all_PID_xlsx_df = pd.concat(df_list[1:], ignore_index=True)
# Optionally, reset the index if needed
all_PID_xlsx_df.reset_index(drop=True, inplace=True)
all_PID_xlsx_df.rename(columns={'PID': 'Participant', 'gesture': 'Gesture_ID', 'hand used': 'Handedness', 'lifted arms off arm rest?': 'Lifted Arms' }, inplace=True)
print(all_PID_xlsx_df.shape)
all_PID_xlsx_df.head()

(270, 22)


,Participant,Gesture_ID,time,description,Handedness,Lifted Arms,dynamic gesture?,muscles activated 1,muscles activated 2,muscles activated 3,...,gesture ideas origin 1,gesture ideas origin 2,gesture ideas origin 3,one hand v 2 hands,one hand v 2 hands.1,notes,muscles activated 4,body part used 1,body part used 2,body part used 3
0,P102,rotate,4,rotate entire right hand from wrist,right,yes,yes,wrist/forearm,shoulder,fingers,...,tablet/touchscreen,NaN,NaN,due to handedness,other hand is too disabled,NaN,NaN,NaN,NaN,NaN
1,P102,move,2,point right hand and move entire arm from left...,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P102,select-single,3,point at object using right index finger,right,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P102,zoom-in,2,use two hands to make a frame and move hands c...,both,yes,yes,shoulder,upper-body (bicep/tricep),fingers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P102,zoom-out,5,use right thumb and index finger and pinch fin...,right,yes,yes,fingers,wrist/forearm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
filtered_metadata_df = all_PID_xlsx_df.loc[:,['Participant','Gesture_ID','Handedness', 'Lifted Arms','muscles activated 1']]
print(filtered_metadata_df.shape)
# make sure same users are in both dfs
filtered_metadata_df = filtered_metadata_df[filtered_metadata_df['Participant'].isin(['P102','P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109', 'P110', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119', 'P121', 'P122', 'P123', 'P125', 'P126',
 'P127', 'P128', 'P132'])]
print(filtered_metadata_df.shape)
print(set(pca_by_signal_df['Participant'].unique()) == set(filtered_metadata_df['Participant'].unique()))

(270, 5)
(230, 5)
True


In [21]:
pca_signal_meta_df = pd.merge(filtered_metadata_df, pca_by_signal_df, on=['Participant', 'Gesture_ID'], how='inner')
pca_column_meta_df = pd.merge(filtered_metadata_df, pca_by_column_df, on=['Participant', 'Gesture_ID'], how='inner')
print(pca_signal_meta_df.shape)
pca_signal_meta_df.head()


(147200, 9)


,Participant,Gesture_ID,Handedness,Lifted Arms,muscles activated 1,Gesture_Num,PC1,PC2,PC3
0,P102,rotate,right,yes,wrist/forearm,1,-0.239026,-0.611281,-0.876869
1,P102,rotate,right,yes,wrist/forearm,1,-0.249440,-0.677231,-0.860271
2,P102,rotate,right,yes,wrist/forearm,1,-0.149920,-0.656097,-0.852871
3,P102,rotate,right,yes,wrist/forearm,1,-0.202984,-0.613159,-0.785642
4,P102,rotate,right,yes,wrist/forearm,1,0.551187,-0.709154,-0.579720


## Make sure labeling is consistent

In [23]:
print(pca_signal_meta_df['Handedness'].unique())
print(pca_signal_meta_df['Lifted Arms'].unique())
print(pca_signal_meta_df['muscles activated 1'].unique())

['right' 'both' 'head' 'left' 'shoulder']
['yes' 'barely' 'no']
['wrist/forearm' 'shoulder' 'fingers' 'upper-body (bicep/tricep)']


## plot based on handedness

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Handedness',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Handedness')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Handedness'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## plot Based on Lifted Arms

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='Lifted Arms',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Lifting Arms')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['Lifted Arms'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        x=0.8,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

## Plot based on primary muscles activated

In [ ]:
fig = px.scatter_3d(pca_signal_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Signal, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()

In [ ]:
fig = px.scatter_3d(pca_column_meta_df, 
                    x='PC1', 
                    y='PC2', 
                    z='PC3', 
                    color='muscles activated 1',
                    title='3D PCA of Biosignal Data, Standardized by Channel, Primary Muscles Activated')

# Adjust marker size for the scatter plot and hide the legend for these traces
fig.update_traces(marker=dict(size=3), showlegend=False)

# Get unique Handedness values and their corresponding colors
unique_handedness = pca_signal_meta_df['muscles activated 1'].unique()
color_map = px.colors.qualitative.Plotly  # Use the same color scheme as Plotly

# Add invisible traces with larger markers for the legend
# I couldnt be bothered to change the var names
for i, handedness in enumerate(unique_handedness):
    fig.add_scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=15, color=color_map[i % len(color_map)]),  # Larger marker size for the legend
        legendgroup=handedness,
        showlegend=True,
        name=handedness
    )

# Update layout
fig.update_layout(
    title=dict(
        font=dict(size=24)
    ), 
    scene=dict(
        xaxis=dict(showticklabels=False, title='PC1'),
        yaxis=dict(showticklabels=False, title='PC2'),
        zaxis=dict(showticklabels=False, title='PC3')
    ),
    width=1200,  # Set the width of the plot
    height=800,  # Set the height of the plot
    margin=dict(l=20, r=20, t=50, b=20),  # Adjust margins
    legend=dict(
        title=dict(text='Primary Muscles Activated'),
        x=0.7,  # Adjust x position of the legend
        y=0.9,  # Adjust y position of the legend
        traceorder='normal',
        font=dict(
            size=16,
        ),
        bgcolor='rgba(255, 255, 255, 0.5)',
        bordercolor='Black',
        borderwidth=1
    )
)

fig.show()